## Code that will be used to generate a data frame from which the averages of various performances accross the data sets can be found

In [1]:
import numpy as np
import pandas as pd

# calculate annualised sharpe ratio
def getAnnualisedSR(APY, Rf, numDays):
    return (APY-Rf)/np.sqrt(numDays)
# calculate cumulative return
# this is just the data since we stored this

# calculate APY
def getAPY(data, years):
    """
    Make sure to pass this years that correspond to the current data set
    E.g 254 days per year on average
    """
    # standard here is two years
    Sn = data[-1]
    return np.power(Sn, 1/years) - 1

# calculate MDD
def getMDD(data):
    series = pd.Series(data)
    rollMax = series.cummax()
    dailyDrawDown = series / rollMax - 1
    return dailyDrawDown.cummin().to_numpy().min()

# group to make a single calculation of the metrics
def getMetrics(data, Rf, years, numDays):
    MDD = getMDD(data)
    APY = getAPY(data, years)
    ASR = getAnnualisedSR(APY, Rf, numDays)
    return MDD, APY, ASR

## Code that will be used to retrieve data

In [2]:
def retrieveAllData(exchangeName, startDate, endDate, length):
    # stack as rows on top of each other
    windowSizes = np.arange(start = 10, stop = 200, step = 10)
    allData = np.empty((len(windowSizes), length))
    count = 0
    for i in windowSizes:
        allData[count,:] = np.loadtxt("./{0}/TrainVal/-TRAINVAL-TrainSize-{1}-Start-{2}-End-{3}".format(exchangeName, i, startDate-i, endDate))
        count += 1
    return allData

## Creation of the data frames

In [3]:

# exchanges
exchanges = ["BIS", "BOV", "JSE", "NAS", "SP5"]

# start days
startDays = np.array([200])

# window sizes
windows = np.arange(start = 10, stop = 200, step = 10)
windows = windows.astype(np.int16)

# end days
endDays = np.array([708])

metricsDF = pd.DataFrame(columns = ["TrainSize", "Exchange", "MDD", "APY", "ASR", "Performance"]) 

## Getting all the data

In [4]:
startDate = np.array([200, 1362])
endDate = np.array([708, 1870])
for exchange in exchanges:
    for count in range(2):
        data = retrieveAllData(exchange, startDate[count], endDate[count], 510)
        windowSize = 10
        for i in range(data.shape[0]):
            MDD, APY, ASR = getMetrics(data[i,:], 0.04, 510/252, 510)
            Performance = MDD + APY + ASR
            metricsDF.loc[len(metricsDF.index)] = [windowSize, exchange, MDD, APY, ASR, Performance]
            windowSize += 10
    

In [5]:
metricData = np.empty((19,3))
trainSizeMetrics = pd.DataFrame(columns = ["TrainSize", "MDD", "APY", "ASR", "Performance"]) 
count = 0

for window in windows:
    tempDF = metricsDF[metricsDF["TrainSize"] == window]
    meanMDD = tempDF.MDD.mean()
    meanAPY = tempDF.APY.mean()
    meanASR = tempDF.ASR.mean()
    meanPerformance = tempDF.Performance.mean()
    trainSizeMetrics.loc[len(trainSizeMetrics.index)] = [window, meanMDD, meanAPY, meanASR, meanPerformance]



In [6]:
display(trainSizeMetrics.head(70))
trainSizeMetrics.to_csv('metricsAcrossExchanges.csv')


,TrainSize,MDD,APY,ASR,Performance
0,10.0,-0.277196,0.664773,0.027665,0.415242
1,20.0,-0.255151,0.547023,0.022451,0.314323
2,30.0,-0.289584,0.672485,0.028007,0.410908
3,40.0,-0.249214,0.599939,0.024794,0.375519
4,50.0,-0.273954,0.638963,0.026523,0.391531
5,60.0,-0.312416,0.632239,0.026225,0.346048
6,70.0,-0.287485,0.650193,0.027020,0.389728
7,80.0,-0.282168,0.652032,0.027101,0.396965
8,90.0,-0.303790,0.594312,0.024545,0.315067
9,100.0,-0.326577,0.644046,0.026748,0.344216


In [7]:
jseDF = metricsDF[metricsDF["Exchange"] == "BIS"]
jseDF.head(70)

,TrainSize,Exchange,MDD,APY,ASR,Performance
0,10,BIS,-0.275264,0.290744,0.011103,0.026583
1,20,BIS,-0.230800,0.409137,0.016346,0.194683
2,30,BIS,-0.634944,0.337434,0.013171,-0.284339
3,40,BIS,-0.375003,0.227495,0.008302,-0.139205
4,50,BIS,-0.205500,0.245424,0.009096,0.049020
5,60,BIS,-0.546121,0.395820,0.015756,-0.134546
6,70,BIS,-0.329108,0.524026,0.021433,0.216352
7,80,BIS,-0.363224,0.289589,0.011052,-0.062583
8,90,BIS,-0.434750,0.341643,0.013357,-0.079750
9,100,BIS,-0.636251,0.219310,0.007940,-0.409001
